In [5]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc4'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

(2156208, 2) (1724990, 2) (1773289, 2) (2224199, 2)
reset_index 완료


,bin,label
0,55,1
1,89,0
2,e5,0
3,53,0
4,83,0


In [6]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [7]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2156208, 257) (1724990, 257) (1773289, 257) (2224199, 257)


In [8]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2151901
1       4307
Name: label, dtype: int64 
 0    1721737
1       3253
Name: label, dtype: int64
0    1770091
1       3198
Name: label, dtype: int64 
 0    2221208
1       2991
Name: label, dtype: int64


In [9]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    322630
ff    186095
8b    155808
45    113507
89    100993
       ...  
a9       269
9e       266
b1       252
96       252
a6       239
Name: bin, Length: 256, dtype: int64

In [10]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
left_idx, right_idx = -10, 11 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 87572


In [11]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 65777


In [12]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 65474


In [13]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 61356


In [14]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(87572, 256) (87572,)
(65777, 256) (65777,)
(65474, 256) (65474,)
(61356, 256) (61356,)


In [15]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(87572, 1, 256) (87572, 1, 1)
(65777, 1, 256) (65777, 1, 1)
(65474, 1, 256) (65474, 1, 1)
(61356, 1, 256) (61356, 1, 1)


In [16]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(280179, 1, 256) (280179, 1, 1)


In [17]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):
    
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
252161/252161 [==============================] - 16s 64us/step - loss: 0.0976 - accuracy: 0.9719
Epoch 2/3
252161/252161 [==============================] - 16s 63us/step - loss: 0.0853 - accuracy: 0.9735
Epoch 3/3
252161/252161 [==============================] - 16s 63us/step - loss: 0.0856 - accuracy: 0.9735


accuracy_score 0.9951102862445571
recall_score 1.0
precision_score 0.9094514210178454
f1_score 0.9525787469712704
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 17s 69us/step - loss: 0.0988 - accuracy: 0.9717
Epoch 2/3
252161/252161 [==============================] - 17s 69us/step - loss: 0.0858 - accuracy: 0.9734
Epoch 3/3
252161/252161 [==============================] - 17s 69us/step - loss: 0.0861 - accuracy: 0.9734


accuracy_score 0.9964665572132201
recall_score 1.0
precision_score 0.9335124244459369
f1_score 0.9656130600903091
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 17s 69us/step - loss: 0.0987 - accuracy: 0.9717
Epoch 2/3
252161/252161 [==============================] - 17s 67us/step - loss: 0.0855 - accuracy: 0.9733
Epoch 3/3
252161/252161 [==============================] - 17s 68us/step - loss: 0.0860 - accuracy: 0.9733


accuracy_score 0.9971446926975516
recall_score 1.0
precision_score 0.9447895100069013
f1_score 0.971611071682044
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 18s 72us/step - loss: 0.0909 - accuracy: 0.9748
Epoch 2/3
252161/252161 [==============================] - 19s 75us/step - loss: 0.0788 - accuracy: 0.9764
Epoch 3/3
252161/252161 [==============================] - 21s 85us/step - loss: 0.0787 - accuracy: 0.9764


accuracy_score 0.9693768291812407
recall_score 0.3889695210449927
precision_score 0.9710144927536232
f1_score 0.555440414507772
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 22s 89us/step - loss: 0.0898 - accuracy: 0.9754
Epoch 2/3
252161/252161 [==============================] - 18s 73us/step - loss: 0.0773 - accuracy: 0.9770
Epoch 3/3
252161/252161 [==============================] - 18s 73us/step - loss: 0.0774 - accuracy: 0.9770


...

accuracy_score 0.9650581768862874
recall_score 0.2947139753801593
precision_score 0.9878640776699029
f1_score 0.4539877300613498
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 19s 75us/step - loss: 0.0913 - accuracy: 0.9752
Epoch 2/3
252161/252161 [==============================] - 18s 73us/step - loss: 0.0780 - accuracy: 0.9769
Epoch 3/3
252161/252161 [==============================] - 19s 74us/step - loss: 0.0786 - accuracy: 0.9769


accuracy_score 0.9646655721322007
recall_score 0.2787125091441112
precision_score 0.9896103896103896
f1_score 0.4349315068493151
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 22s 87us/step - loss: 0.0908 - accuracy: 0.9751
Epoch 2/3
252161/252161 [==============================] - 21s 82us/step - loss: 0.0784 - accuracy: 0.9767
Epoch 3/3
252161/252161 [==============================] - 27s 106us/step - loss: 0.0783 - accuracy: 0.9767


accuracy_score 0.9662359911485474
recall_score 0.3117261471230881
precision_score 0.9976689976689976
f1_score 0.4750277469478357
======Training stage======
Epoch 1/3
252161/252161 [==============================] - 21s 85us/step - loss: 0.0913 - accuracy: 0.9747
Epoch 2/3
252161/252161 [==============================] - 20s 78us/step - loss: 0.0788 - accuracy: 0.9764
Epoch 3/3
252161/252161 [==============================] - 20s 79us/step - loss: 0.0788 - accuracy: 0.9764


accuracy_score 0.969519594546363
recall_score 0.37738619676945667
precision_score 0.9884615384615385
f1_score 0.5462274176408076
======Training stage======
Epoch 1/3
252162/252162 [==============================] - 20s 80us/step - loss: 0.0914 - accuracy: 0.9748
Epoch 2/3
252162/252162 [==============================] - 22s 87us/step - loss: 0.0792 - accuracy: 0.9763
Epoch 3/3
252162/252162 [==============================] - 21s 82us/step - loss: 0.0792 - accuracy: 0.9763


accuracy_score 0.9698040475425634
recall_score 0.38179148311306904
precision_score 0.9923664122137404
f1_score 0.5514316012725345

K-fold cross validation Accuracy: [0.9951102862445571, 0.9964665572132201, 0.9971446926975516, 0.9693768291812407, 0.9633449925048183, 0.9650581768862874, 0.9646655721322007, 0.9662359911485474, 0.969519594546363, 0.9698040475425634]

K-fold cross validation Recall: [1.0, 1.0, 1.0, 0.3889695210449927, 0.2645998558038933, 0.2947139753801593, 0.2787125091441112, 0.3117261471230881, 0.37738619676945667, 0.38179148311306904]

K-fold cross validation Precision: [0.9094514210178454, 0.9335124244459369, 0.9447895100069013, 0.9710144927536232, 0.9812834224598931, 0.9878640776699029, 0.9896103896103896, 0.9976689976689976, 0.9884615384615385, 0.9923664122137404]

K-fold cross validation F1-Score: [0.9525787469712704, 0.9656130600903091, 0.971611071682044, 0.555440414507772, 0.41680863145939806, 0.4539877300613498, 0.4349315068493151, 0.4750277469478357, 0.5462274176

In [18]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9756726740097349
10-Fold Cross_validation. Recall : 0.529789968837877
10-Fold Cross_validation. Precision : 0.969602268630877
10-Fold Cross_validation. F1-Score : 0.6323657927482637
